In [59]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [60]:
# Import a bunch of libraries.
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from plotnine import *
import plotly.express as px
import plotly.io as pio
import folium

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import altair as alt

# display multiple outputs from a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# load modules to support web scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup

# support output to HTML
from IPython.display import Javascript
from IPython.core.display import display, HTML

In [61]:
#VULNERABLE - COUNTIES - PRESENTATION - START
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import altair as alt # For displaying graphs
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import panel as pn
import datetime as dt
from altair import datum
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import requests
for dirname, _, filenames in os.walk('./icu-data/'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        #print(file)

# Any results you write to the current directory are saved as output.

# **Data that specifies ICU Beds by State and County. It also factors in the Seniors (60+) in the County**

In [62]:
demoGraphicAndICUByCounty = pd.read_csv(file)
#print("Summary of ICU Beds per State and County with Residents (60+) per ICU Bed")
demoGraphicAndICUByCounty = demoGraphicAndICUByCounty.rename(columns = {"ICU Beds":"ICU_Beds","Total Population":"Total_Population","Population Aged 60+":"Population_Aged_60+","Percent of Population Aged 60+":"Percent_Population_Aged_60","Residents Aged 60+ Per Each ICU Bed":"Residents_Per_ICU_Bed"})
#demoGraphicAndICUByCounty

# **Data that specifies US County wise daily COVID19 count. This is a time series data account from 01 / Jan /2020)**

In [63]:
jhuDailyDataSet=pd.read_csv('./us-counties/us-counties.csv')
#print("Summary of JHU Daily Dataset across the US (Starting Jan 01 2020)")
#jhuDailyDataSet

In [64]:
countWithNoICUBeds = demoGraphicAndICUByCounty.query("ICU_Beds==0")
#countWithNoICUBeds
countiesWithICUBeds = demoGraphicAndICUByCounty.query("ICU_Beds>0")

## BRIEF TEXT SUMMARY

In [65]:
comp = pd.read_csv('data/comp.csv', parse_dates=['Start', 'End'])

## Into The World of Pandemics... A Comparison

In this section, we will compare and contrast the five major pandemics in the 21st century.  These pandemics are:

* Covid-19 of 2019
* Ebola of 2014
* MERS of 2012
* H1N1 of 2009
* SARS of 2003

### Overview

The interactive chart below shows the recent five major pandemics in the 21st century along with the seaonsal flu.  Place your cursor over any colored bar to learn more about each pandemic.

In [66]:
pandemicChart = alt.Chart(comp).mark_bar(size=10.0).encode(
    x = 'Start:T',
    x2 = 'End:T',
    y = 'Epidemic:N',
    color = 'Epidemic',
    tooltip = ['Epidemic', 'Key Symptoms', 'First Detection', 'Global Cases', 'Global Deaths',
               'Transmission', 'Most Affected Groups', 'Treatments Available', 'Vaccines Available',
               'End of Epidemic']
)

In [67]:
pandemicChart.interactive()

alt.Chart(...)

## COVID-19 DATALOAD

In [68]:
# COVID-19
# --------

# covid_19 dataset
covid_19 = pd.read_csv('data/covid_19_clean_complete.csv', parse_dates=['Date'])

# selecting important columns only
covid_19 = covid_19[['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered']]

# replacing Mainland china with just China
covid_19['Country/Region'] = covid_19['Country/Region'].replace('Mainland China', 'China')

# renaming columns
covid_19.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']

# group by date and country
covid_19 = covid_19.groupby(['Date', 'Country'])['Cases', 'Deaths', 'Recovered']
covid_19 = covid_19.sum().reset_index()

# latest
c_lat = covid_19[covid_19['Date'] == max(covid_19['Date'])].reset_index()

# latest grouped by country
c_lat_grp = c_lat.groupby('Country')['Cases', 'Deaths', 'Recovered'].sum().reset_index()
c_lat_grp['sqrtCases'] = np.sqrt(c_lat_grp['Cases'])

# nth day
covid_19['nth_day'] = (covid_19['Date'] - min(covid_19['Date'])).dt.days

# day by day
c_dbd = covid_19.groupby('Date')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
c_dbd['nth_day'] = covid_19.groupby('Date')['nth_day'].max().values

# mortality rate
c_dbd['Mortality'] = c_dbd['Deaths'] / c_dbd['Cases'] * 100

# no. of countries
temp = covid_19[covid_19['Cases']>0]
c_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## EBOLA DATALOAD

In [69]:
# EBOLA
# ------

# ebola dataset
ebola_14 = pd.read_csv("data/ebola_2014_2016_clean.csv", parse_dates=['Date'])

# selecting important columns only
ebola_14 = ebola_14[['Date', 'Country', 'No. of confirmed, probable and suspected cases',
                     'No. of confirmed, probable and suspected deaths']]

# renaming columns
ebola_14.columns = ['Date', 'Country', 'Cases', 'Deaths']

In [70]:
# temp = ebola_14.copy()

# for c in ebola_14['Country'].unique():
#     temp_data = ebola_14[ebola_14['Country'] == c]
#     idx = ebola_14.index[ebola_14['Country'] == c]

#     for i in range(len(idx)-1):        
#         if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
#             tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
#             while tracker < temp_data.loc[idx[i+1]]['Date']:
#                 cases = temp_data.loc[idx[i]]['Cases']
#                 deaths = temp_data.loc[idx[i]]['Deaths']
#                 to_add = pd.DataFrame([{'Date': tracker, 
#                                        'Country': c, 
#                                        'Cases': cases, 
#                                        'Deaths': deaths}])
#                 temp = temp.append(to_add, ignore_index = True)
#                 tracker += timedelta(days=1)

# ebola_14 = temp

In [71]:
# group by date and country
ebola_14 = ebola_14.groupby(['Date', 'Country'])['Cases', 'Deaths']
ebola_14 = ebola_14.sum().reset_index()

# filling missing values
ebola_14['Cases'] = ebola_14['Cases'].fillna(0)
ebola_14['Deaths'] = ebola_14['Deaths'].fillna(0)

# converting datatypes
ebola_14['Cases'] = ebola_14['Cases'].astype('int')
ebola_14['Deaths'] = ebola_14['Deaths'].astype('int')

# latest
e_lat = ebola_14[ebola_14['Date'] == max(ebola_14['Date'])].reset_index()

# latest grouped by country
e_lat_grp = e_lat.groupby('Country')['Cases', 'Deaths'].sum().reset_index()

# nth day
ebola_14['nth_day'] = (ebola_14['Date'] - min(ebola_14['Date'])).dt.days

# day by day
e_dbd = ebola_14.groupby('Date')['Cases', 'Deaths'].sum().reset_index()

# nth day
e_dbd['nth_day'] = ebola_14.groupby('Date')['nth_day'].max().values

# mortality rate
e_dbd['Mortality'] = e_dbd['Deaths'] / e_dbd['Cases'] * 100

# no. of countries
temp = ebola_14[ebola_14['Cases']>0]
e_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

ebola_14['Recovered'] = 0
ebola_14 = ebola_14[['Date','Country','Cases','Deaths','Recovered','nth_day']]

## SARS DATALOAD

In [72]:
# SARS
# ----

# sars dataset
sars_03 = pd.read_csv("data/sars_2003_complete_dataset_clean.csv", parse_dates=['Date'])

# selecting important columns only
sars_03 = sars_03[['Date', 'Country', 'Cumulative number of case(s)', 
                   'Number of deaths', 'Number recovered']]

# renaming columns
sars_03.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']

# group by date and country
sars_03 = sars_03.groupby(['Date', 'Country'])['Cases', 'Deaths', 'Recovered']
sars_03 = sars_03.sum().reset_index()

In [73]:
temp = sars_03.copy()

for c in sars_03['Country'].unique():
    temp_data = sars_03[sars_03['Country'] == c]
    idx = sars_03.index[sars_03['Country'] == c]

    for i in range(len(idx)-1):        
        if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
            tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
            while tracker < temp_data.loc[idx[i+1]]['Date']:
                cases = temp_data.loc[idx[i]]['Cases']
                deaths = temp_data.loc[idx[i]]['Deaths']
                recovered = temp_data.loc[idx[i]]['Recovered']
                to_add = pd.DataFrame([{'Date': tracker, 
                                       'Country': c, 
                                       'Cases': cases, 
                                       'Deaths': deaths,
                                       'Recovered': recovered}])
                temp = temp.append(to_add, ignore_index = True)
                tracker += timedelta(days=1)

sars_03 = temp

In [74]:
# latest
s_lat = sars_03[sars_03['Date'] == max(sars_03['Date'])].reset_index()

# latest grouped by country
s_lat_grp = s_lat.groupby('Country')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
sars_03['nth_day'] = (sars_03['Date'] - min(sars_03['Date'])).dt.days

# day by day
s_dbd = sars_03.groupby('Date')['Cases', 'Deaths', 'Recovered'].sum().reset_index()

# nth day
s_dbd['nth_day'] = sars_03.groupby('Date')['nth_day'].max().values

# mortality rate
s_dbd['Mortality'] = s_dbd['Deaths'] / s_dbd['Cases'] * 100

# no. of countries
temp = sars_03[sars_03['Cases']>0]
s_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## H1N1 DATALOAD

In [75]:
# H1N1
# ----

# h1n1 dataset
h1n1_09 = pd.read_csv("data/Pandemic (H1N1) 2009.csv", parse_dates=['Update Time'])

# retain the date portion of 'Update Time'
h1n1_09['Date'] = h1n1_09['Update Time'].dt.date

# keep relevant columns
h1n1_09 = h1n1_09[['Date', 'Country', 'Cases', 'Deaths']]

In [76]:
temp = h1n1_09.copy()

for c in h1n1_09['Country'].unique():
    temp_data = h1n1_09[h1n1_09['Country'] == c]
    idx = h1n1_09.index[h1n1_09['Country'] == c]

    for i in range(len(idx)-1):        
        if ((temp_data.loc[idx[i+1]]['Date'] - temp_data.loc[idx[i]]['Date']) >= timedelta(days=2)):
            tracker = temp_data.loc[idx[i]]['Date'] + timedelta(days=1)  
            
            while tracker < temp_data.loc[idx[i+1]]['Date']:
                cases = temp_data.loc[idx[i]]['Cases']
                deaths = temp_data.loc[idx[i]]['Deaths']
                to_add = pd.DataFrame([{'Date': tracker, 
                                       'Country': c, 
                                       'Cases': cases, 
                                       'Deaths': deaths}])
                temp = temp.append(to_add, ignore_index = True)
                tracker += timedelta(days=1)

h1n1_09 = temp

In [77]:
# latest
h_lat = h1n1_09[h1n1_09['Date'] == max(h1n1_09['Date'])].reset_index()

# latest grouped by country
h_lat_grp = h_lat.groupby('Country')['Cases', 'Deaths'].sum().reset_index()

# nth day calculation and reverse the row order
h1n1_09['nth_day'] = (h1n1_09['Date'] - min(h1n1_09['Date'])).dt.days
h1n1_09 = h1n1_09.iloc[::-1].reset_index(drop=True)

# fill NaN with 0
h1n1_09 = h1n1_09.fillna(0)

# change 'Deaths' column to integers
h1n1_09['Deaths'] = h1n1_09['Deaths'].astype(int)

# day by day
h_dbd = h1n1_09.groupby('Date')['Cases', 'Deaths'].sum().reset_index()

# nth day
h_dbd['nth_day'] = h1n1_09.groupby('Date')['nth_day'].max().values

# mortality rate
h_dbd['Mortality'] = h_dbd['Deaths'] / h_dbd['Cases'] * 100

# no. of countries
temp = h1n1_09[h1n1_09['Cases']>0]
h_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## MERS DATALOAD

In [78]:
# MERS
# ------

# mers dataset
mers_12 = pd.read_csv("data/mers_weekly_clean.csv")
mers_cntry = pd.read_csv("data/mers_country_count_latest.csv")

# transforming data to obtain 'Date'
mers_12['Date'] = mers_12.apply(lambda row: str(row['Year']) + '-W' + str(row['Week']) +'-1', axis=1)
test = datetime.datetime.strptime('2012-W12-1', '%Y-W%W-%w')
mers_12['Date'] = mers_12.apply(lambda row: datetime.datetime.strptime(row['Date'], '%Y-W%W-%w'), axis=1)

# keeping important columns only
mers_12 = mers_12[['Region', 'New Cases', 'Date']]

# renaming and rearranging columns
mers_12.columns = ['Country', 'Cases', 'Date']
mers_12 = mers_12[['Date', 'Country', 'Cases']]

# calculate cumulative cases
temp = pd.DataFrame({'Date':[], 'Country':[], 'Cases':[]})

for c in mers_12.Country.unique():
    temp_df = mers_12[mers_12['Country']==c]
    temp_df['Cases'] = temp_df['Cases'].cumsum()
    temp = temp.append(temp_df, ignore_index = True)
    
mers_12 = temp

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
# latest
m_lat = mers_12[mers_12['Date'] == max(mers_12['Date'])].reset_index()

# latest grouped by country
m_lat_grp = m_lat.groupby('Country')['Cases'].sum().reset_index()

# nth day
mers_12['nth_day'] = (mers_12['Date'] - min(mers_12['Date'])).dt.days

# day by day
m_dbd = mers_12.groupby('Date')['Cases'].sum().reset_index()

# nth day
m_dbd['nth_day'] = mers_12.groupby('Date')['nth_day'].max().values

# # no. of countries
# temp = sars_03[sars_03['Cases']>0]
# s_dbd['n_countries'] = temp.groupby('Date')['Country'].apply(len).values

## MERGING DATA

In [80]:
e_dbd['new_cases'] = e_dbd['Cases'].diff()
c_dbd['new_cases'] = c_dbd['Cases'].diff()
s_dbd['new_cases'] = s_dbd['Cases'].diff()
h_dbd['new_cases'] = h_dbd['Cases'].diff()
m_dbd['new_cases'] = m_dbd['Cases'].diff()

e_dbd['new_deaths'] = e_dbd['Deaths'].diff()
c_dbd['new_deaths'] = c_dbd['Deaths'].diff()
s_dbd['new_deaths'] = s_dbd['Deaths'].diff()
h_dbd['new_deaths'] = h_dbd['Deaths'].diff()

e_dbd['epidemic'] = 'EBOLA'
c_dbd['epidemic'] = 'COVID-19'
s_dbd['epidemic'] = 'SARS'
h_dbd['epidemic'] = 'H1N1'
m_dbd['epidemic'] = 'MERS'

In [81]:
c_lat_grp["log_Cases"] = np.log(c_lat_grp["Cases"])
e_lat_grp["log_Cases"] = np.log(e_lat_grp["Cases"])
s_lat_grp["log_Cases"] = np.log(s_lat_grp["Cases"])
h_lat_grp["log_Cases"] = np.log(h_lat_grp["Cases"])

c_lat_grp["log_Deaths"] = np.log(c_lat_grp["Deaths"]+1)
e_lat_grp["log_Deaths"] = np.log(e_lat_grp["Deaths"]+1)
s_lat_grp["log_Deaths"] = np.log(s_lat_grp["Deaths"]+1)
h_lat_grp["log_Deaths"] = np.log(h_lat_grp["Deaths"]+1)

In [82]:
covid_19['Epidemic'] = 'COVID-19'
ebola_14['Epidemic'] = 'EBOLA'
sars_03['Epidemic'] = 'SARS'
h1n1_09['Epidemic'] = 'H1N1'
mers_12['Epidemic'] = 'MERS'

In [83]:
data = pd.concat([c_dbd, s_dbd, e_dbd, h_dbd, m_dbd])
data.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,Cases,Date,Deaths,Mortality,Recovered,epidemic,n_countries,new_cases,new_deaths,nth_day
0,555.0,2020-01-22 00:00:00,17.0,3.063063,28.0,COVID-19,6.0,NaN,NaN,0
1,654.0,2020-01-23 00:00:00,18.0,2.752294,30.0,COVID-19,8.0,99.0,1.0,1
2,941.0,2020-01-24 00:00:00,26.0,2.763018,36.0,COVID-19,9.0,287.0,8.0,2
3,1434.0,2020-01-25 00:00:00,42.0,2.928870,39.0,COVID-19,11.0,493.0,16.0,3
4,2118.0,2020-01-26 00:00:00,56.0,2.644004,52.0,COVID-19,13.0,684.0,14.0,4


In [84]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

### Spread Across the Globe

Scroll to the world map of the pandemic of interest.  Place mouse over the country to learn the number of reported cases.  With the mouse over the world map, scroll up and down to zoom in and out.

In [85]:
covidFig = px.choropleth(c_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='COVID-19 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

ebolaFig = px.choropleth(e_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='EBOLA 2014 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

sarsFig = px.choropleth(s_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='SARS 2003 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

h1n1Fig = px.choropleth(h_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='H1N1 2009 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)


#-----------------------------------------------------------------------------------------

mersFig = px.choropleth(m_lat_grp, locations="Country", locationmode='country names',
                    color="Cases", hover_name="Country", 
                    color_continuous_scale="reds", title='MERS 2012 REPORTED CASES')
# fig.update(layout_coloraxis_showscale=False)

In [86]:
covidFig.show()

In [87]:
ebolaFig.show()

In [88]:
sarsFig.show()

In [89]:
h1n1Fig.show()

In [90]:
mersFig.show()

### Reported Cases By Day

The line chart on the next slide shows the number of reported cases of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [91]:
interval_cases = alt.selection_interval(encodings=['x'])

base_cases = alt.Chart(data).mark_line(clip=True).encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Cases:Q',
    color = 'epidemic'
)

chart_cases = base_cases.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_cases.ref()))
)

view_cases = base_cases.properties(
    width=800,
    height=50,
    selection=interval_cases
)

In [92]:
chart_cases & view_cases

alt.VConcatChart(...)

### Reported Deaths By Day

The line chart on the next slide shows the number of reported deaths of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [93]:
interval_deaths = alt.selection_interval(encodings=['x'])

base_deaths = alt.Chart(data).mark_line().encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Deaths:Q',
    color = 'epidemic'
)

chart_deaths = base_deaths.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_deaths.ref()))
)

view_deaths = base_deaths.properties(
    width=800,
    height=50,
    selection=interval_deaths
)

In [94]:
chart_deaths & view_deaths

alt.VConcatChart(...)

### Mortality Rate By Day

The line chart on the next slide shows the mortality rate of each pandemic by day.  The view window (narrow window on the bottom) is interactive.  Double click on the view window and drag the mouse to select a timeframe of interest.  Click and hold the mouse and drag left and right to slide the selected timeframe.  With the mouse over the selected timeframe, scroll up and down to widen and shrink the selected timeframe.

In [95]:
interval_deaths = alt.selection_interval(encodings=['x'])

base_deaths = alt.Chart(data).mark_line().encode(
    x = alt.X('nth_day:Q', scale=alt.Scale(domain=(0,600))),
    y = 'Mortality:Q',
    color = 'epidemic'
)

chart_deaths = base_deaths.properties(
    width=800,
    height=300
).encode(
    x=alt.X('nth_day:Q', scale=alt.Scale(domain=interval_deaths.ref()))
)

view_deaths = base_deaths.properties(
    width=800,
    height=50,
    selection=interval_deaths
)

In [96]:
chart_deaths & view_deaths

alt.VConcatChart(...)

### Reported Cases By Country

Scroll to the tree map of the pandemic of interest.  Place mouse over the country to learn the number of reported cases.

In [97]:
import plotly.io as pio

covidFig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='COVID-19 Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# fig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
#                  path=["Country"], values="sqrtCases", title='COVID-19',
#                  color_discrete_sequence = px.colors.qualitative.Set1)
# fig.show()

# pio.write_html(covidFig, file='index_covid_19_cases.html', auto_open=False)

ebolaFig = px.treemap(e_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='EBOLA Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(ebolaFig, file='index_ebola_14_cases.html', auto_open=False)

sarsFig = px.treemap(s_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='SARS Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(sarsFig, file='index_sars_03_cases.html', auto_open=False)

h1n1Fig = px.treemap(h_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Cases", title='H1N1 Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(h1n1Fig, file='index_h1n1_09_cases.html', auto_open=False)

mersFig = px.treemap(mers_cntry, 
                 path=["Country"], values="Confirmed", title='MERS Cases',
                 color_discrete_sequence = px.colors.qualitative.Set1)

# pio.write_html(mersFig, file='index_mers_12_cases.html', auto_open=False)

In [98]:
covidFig.show()

In [99]:
ebolaFig.show()

In [100]:
sarsFig.show()

In [101]:
h1n1Fig.show()

In [102]:
mersFig.show()

### Reported Deaths By Country

Scroll to the tree map of the pandemic of interest.  Place mouse over the country to learn the number of reported deaths.

In [103]:
covidFig = px.treemap(c_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='COVID-19 Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# fig = px.treemap(c_lat_grp.sort_values(by='Cases', ascending=False).reset_index(drop=True), 
#                  path=["Country"], values="sqrtCases", title='COVID-19',
#                  color_discrete_sequence = px.colors.qualitative.Set1)
# fig.show()

# pio.write_html(covidFig, file='index_covid_19_deaths.html', auto_open=False)

ebolaFig = px.treemap(e_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='EBOLA Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(ebolaFig, file='index_ebola_14_deaths.html', auto_open=False)

sarsFig = px.treemap(s_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='SARS Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(sarsFig, file='index_sars_03_deaths.html', auto_open=False)

h1n1Fig = px.treemap(h_lat_grp.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country"], values="Deaths", title='H1N1 Deaths',
                 color_discrete_sequence = px.colors.qualitative.Set1)


# pio.write_html(h1n1Fig, file='index_h1n1_09_deaths.html', auto_open=False)

mersFig = px.treemap(mers_cntry, 
                 path=["Country"], values="Confirmed", title='MERS',
                 color_discrete_sequence = px.colors.qualitative.Set1)

# pio.write_html(mersFig, file='index_mers_12.html', auto_open=True)

In [104]:
covidFig.show()

In [105]:
ebolaFig.show()

In [106]:
sarsFig.show()

In [107]:
h1n1Fig.show()

In [108]:
mersFig.show()

### COVID-19 Vulnerable Counties in USA - In this interactive chart series we are going to see how ICU beds are distributed across Counties. Goal is to show the most vulnerable counties having least to no ICU beds for the population it is serving. 

## Most Vulnerable Counties having no ICU Beds in US

In [109]:
#print("Most Vulnerable Counties having No ICU Beds")
chart = alt.Chart(countWithNoICUBeds).mark_circle(size=50).encode(
    x='Total_Population',
    y='Population_Aged_60+',
    color='State',
    tooltip=['State', 'County','Percent_Population_Aged_60','Population_Aged_60+','Total_Population']
).properties(
    title='Vulnerable Counties in US having Zero ICU Bed per County'
)
chart.configure_title(
    fontSize=20,
    font='Arial',
    anchor='start',
    color='gray'
)

alt.Chart(...)

## Vulnerable Counties in US having atleast one ICU Bed, ratio of Seniors to population along with bed to individuals ratio

In [110]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('json')

In [111]:

jhuDailyDataSet = jhuDailyDataSet.rename(columns={'state':'State','county':'County'})
jhuAggregateDataSet = jhuDailyDataSet.groupby(['State','County'])
jhuAggregateDataSetAggregate = jhuAggregateDataSet.agg(np.sum)
#jhuAggregateDataSetAggregate
mergedDataSet = pd.merge(jhuAggregateDataSetAggregate,countiesWithICUBeds,left_on='State',right_on='State')

chart2 = alt.Chart(mergedDataSet).mark_circle(size=50).encode(
    x='cases',
    y='Residents_Per_ICU_Bed',
    color='State',
    tooltip=['State', 'County','Percent_Population_Aged_60','Population_Aged_60+','Total_Population','cases','deaths','Residents_Per_ICU_Bed']
).properties(
    title='Vulnerable Counties in US having atleast One ICU Bed per County'
)

chart2.configure_title(
    fontSize=20,
    font='Arial',
    anchor='start',
    color='gray'
)

alt.Chart(...)